<a href="https://colab.research.google.com/github/ckdlsqpf/projectmanager/blob/main/KLUE_BERT_%EC%9D%B4%EC%A7%84%ED%9D%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate evaluate

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

from transformers import AutoModelForSequenceClassification
seq_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base", num_labels=5,  # [TODO] 레이블 종류의 개수를 설정
    id2label={
        0: '불쾌', 1: '좋음', 2: '놀람', 3: '감정없음', 4: '슬픔'})  # [TODO] 레이블 번호별 설명
seq_model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

데이터

In [8]:
import pandas as pd
df = pd.read_excel('플립.xlsx') # [TODO] 자신의 파일을 불러오기
df.head()

,content,Label
0,내 소원은 줄리 베이커가 내게 관심을 끊는 거였다,0
1,심장이 멎는 듯했다,1
2,그 애는 나를 좋아하는 게 분명한데 수줍음이 많았다,1
3,어린애가 어떻게 그런 생각을 하겠어?,2
4,2학년이 되기 전 1957년 여름에 모든 게 시작됐다,3


In [10]:
from datasets import Dataset

# [TODO]
# 자신의 데이터에서 사용할 컬럼을 선택(예: title)
# 데이터에 맞춰서 row.document -> row.title
def gen():
    for row in df.itertuples():
        yield {'text': row.content, 'label': row.Label}

ds = Dataset.from_generator(gen)

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
ds[0]

{'text': '내 소원은 줄리 베이커가 내게 관심을 끊는 거였다', 'label': 0}

전처리

In [13]:
def convert_example(example):
    result = tokenizer(example['text'])
    result['label'] = [example['label']]
    return result

train_dataset = ds.map(convert_example)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

훈련

In [14]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch")

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=seq_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    data_collator=data_collator
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.362118


TrainOutput(global_step=5, training_loss=1.6220212936401368, metrics={'train_runtime': 1.8311, 'train_samples_per_second': 18.021, 'train_steps_per_second': 2.731, 'total_flos': 324272554482.0, 'train_loss': 1.6220212936401368, 'epoch': 1.0})

저장

In [ ]:
model_path = 'kogpt2-nsmc'
seq_model.save_pretrained(model_path)

사용

In [15]:
from transformers import pipeline
text_cls = pipeline('text-classification', model=seq_model,
              tokenizer=tokenizer, device='cuda:0')

In [22]:
text_cls('나몽이는 탭스 점수가 모자라서 속상하다')

[{'label': '슬픔', 'score': 0.22759482264518738}]